# IMPORT LIBRARIES

In [2]:
import pandas as pd
import random
from pyomo.environ import *
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [3]:
# 1. Build the binary order–SKU matrix R
def build_R(df, order_col='Num. Ordine', sku_col='Articolo'):
    df_unique = df[[order_col, sku_col]].drop_duplicates()
    orders = df_unique[order_col].unique()
    skus = df_unique[sku_col].unique()
    order_idx = pd.Series(np.arange(len(orders)), index=orders)
    sku_idx = pd.Series(np.arange(len(skus)), index=skus)
    rows = df_unique[order_col].map(order_idx).values
    cols = df_unique[sku_col].map(sku_idx).values
    data = np.ones(len(df_unique), dtype=int)
    R = coo_matrix((data, (rows, cols)), shape=(len(orders), len(skus)))
    return R, orders, skus

In [4]:
import numpy as np
from scipy.sparse import coo_matrix

def greedy_bestsellers_volume(R, k_vol, sku_vols, debug=False):
    """
    Algorithm 4: BESTSELLERS, adapted for volume constraints, with safe fallback.
    R        : sparse order×SKU matrix (COO or CSR)
    k_vol    : array of DC capacities in volume
    sku_vols : array of SKU volumes (aligned to R’s columns)
    """
    # Ensure CSR format for slicing
    R_csr = R.tocsr()
    n_orders, n_skus = R_csr.shape
    D = len(k_vol)

    # Compute co-occurrence and per-SKU sales
    C = (R_csr.T).dot(R_csr).toarray()
    sales = np.diag(C)
    sorted_sales = np.argsort(-sales)

    # Convert volume caps to cardinal caps (approx) for B
    avg_vol = np.mean(sku_vols)
    k_card = np.floor(k_vol / avg_vol).astype(int)

    # Active DCs, allocations, remaining volume & SKUs
    active = list(range(D))
    allocation = [set() for _ in range(D)]
    remaining = k_vol.copy()
    remaining_skus = set(range(n_skus))

    # Helper to compute B
    def compute_B(dcs):
        if len(dcs) < 2:
            return 0
        return int((sum(k_card[d] for d in dcs) - n_skus) // (len(dcs) - 1))

    # 1) Phase-0: seed any DC with k_card[d] < B
    B = compute_B(active)
    if debug: print("Initial B =", B)
    while True:
        small = [d for d in active if k_card[d] < B]
        if not small:
            break
        for d in small:
            to_seed = k_card[d]
            seeded = 0
            for s in sorted_sales:
                if s in remaining_skus and sku_vols[s] <= remaining[d]:
                    allocation[d].add(s)
                    remaining[d] -= sku_vols[s]
                    remaining_skus.remove(s)
                    seeded += 1
                    if seeded >= to_seed:
                        break
            active.remove(d)
            if debug:
                print(f"  DC_{d+1} seeded {seeded} SKUs (k_card < B)")
        B = compute_B(active)
        if debug: print("  Recomputed B =", B)

    # 2) Fallback: if B <= 0, call GREEDY_SEEDS on remaining
    if B <= 0:
        if debug: print("B <= 0, falling back to GREEDY_SEEDS")
        # slice R and vols to only remaining SKUs
        idx = sorted(remaining_skus)
        R_sub = R_csr[:, idx]
        vols_sub = sku_vols[idx]
        caps_sub = remaining.copy()
        alloc_sub, _ = greedy_seeds_volume(R_sub, caps_sub, vols_sub, debug=debug)
        # merge sub‐alloc back into main allocation
        for sub_dc, d in enumerate(active):
            for rel_s in alloc_sub[sub_dc]:
                s = idx[rel_s]
                allocation[d].add(s)
        return allocation, remaining

    # 3) Main branch B > 0: seed each active DC with top B SKUs
    if debug: print(f"B > 0, seeding top {B} SKUs to each DC")
    for d in sorted(active, key=lambda x: -k_vol[x]):
        seeded = 0
        for s in sorted_sales:
            if s in remaining_skus and sku_vols[s] <= remaining[d]:
                allocation[d].add(s)
                remaining[d] -= sku_vols[s]
                remaining_skus.remove(s)
                seeded += 1
                if seeded >= B:
                    break
        if debug:
            print(f"  DC_{d+1} seeded {seeded} SKUs")

    # 4) Assign rest by max average co-appearance
    if debug: print("Assigning remaining SKUs by avg co-appearance")
    for s in sorted_sales:
        if s not in remaining_skus:
            continue
        best_d, best_score = None, -np.inf
        for d in active:
            if sku_vols[s] > remaining[d]:
                continue
            alloc_list = list(allocation[d])
            score = np.mean([C[s, x] for x in alloc_list]) if alloc_list else 0.0
            if score > best_score:
                best_score, best_d = score, d
        if best_d is not None:
            allocation[best_d].add(s)
            remaining[best_d] -= sku_vols[s]
            remaining_skus.remove(s)
            if debug:
                print(f"  SKU {s} -> DC_{best_d+1} (avg coappear {best_score:.1f})")

    # 5) Fill any leftover capacity by total co-appearance
    if debug: print("Filling remaining capacity by total co-appearance")
    while remaining_skus:
        progress = False
        for d in active:
            cap = remaining[d]
            cand = [s for s in remaining_skus if sku_vols[s] <= cap]
            if not cand:
                continue
            # total co-appearance score
            if allocation[d]:
                scores = C[list(allocation[d]), :].sum(axis=0)
            else:
                scores = C.sum(axis=0)
            # pick best candidate
            best_s = max(cand, key=lambda s: scores[s])
            allocation[d].add(best_s)
            remaining[d] -= sku_vols[best_s]
            remaining_skus.remove(best_s)
            progress = True
            if debug:
                print(f"  DC_{d+1} added SKU {best_s} (score {scores[best_s]:.0f})")
        if not progress:
            if debug:
                print("  No further allocations possible.")
            break

    return allocation, remaining


In [5]:
import numpy as np

def greedy_seeds_volume(R, k_volume, sku_volumes, debug=False):
    """
    Volume-aware GREEDY SEEDS heuristic with guards against empty candidate sets.
    R: scipy.sparse CSR/COO order×SKU matrix
    k_volume: array of DC capacities in volume
    sku_volumes: array of SKU volumes
    """
    R_csr = R.tocsr()
    n_orders, n_skus = R_csr.shape
    n_dcs = len(k_volume)

    # Compute co-occurrence matrix
    C = (R_csr.T).dot(R_csr).toarray()
    sales = np.diag(C)
    sku_by_sales = np.argsort(-sales)
    sorted_dcs = np.argsort(-k_volume)

    allocation = [set() for _ in range(n_dcs)]
    remaining = k_volume.copy()
    allocated = set()

    # Seed DCs
    # DC 1: top selling SKU
    s0 = sku_by_sales[0]
    if remaining[sorted_dcs[0]] >= sku_volumes[s0]:
        allocation[sorted_dcs[0]].add(s0)
        remaining[sorted_dcs[0]] -= sku_volumes[s0]
        allocated.add(s0)
        if debug:
            print(f"Seed DC_{sorted_dcs[0]+1} with SKU {s0}")

    # Remaining DC seeds
    for d in sorted_dcs[1:]:
        # top decile
        top_n = max(1, int(0.1 * n_skus))
        top_decile = set(sku_by_sales[:top_n])

        # candidates: unallocated, in decile, fits volume
        cand = [s for s in sku_by_sales if s not in allocated 
                and s in top_decile 
                and sku_volumes[s] <= remaining[d]]
        if not cand:
            # fallback to any unallocated that fits
            cand = [s for s in sku_by_sales if s not in allocated and sku_volumes[s] <= remaining[d]]
        if not cand:
            if debug: print(f"No seed candidates for DC_{d+1}; skipping")
            continue
        # choose least coappearance
        scores = [sum(C[s, list(allocated)]) for s in cand] or [0]
        s_seed = cand[int(np.argmin(scores))]
        allocation[d].add(s_seed)
        remaining[d] -= sku_volumes[s_seed]
        allocated.add(s_seed)
        if debug:
            print(f"Seed DC_{d+1} with SKU {s_seed}")

    # Assign remaining SKUs by avg coappearance
    for s in sku_by_sales:
        if s in allocated: continue
        best_d, best_score = None, -np.inf
        for d in sorted_dcs:
            if sku_volumes[s] > remaining[d]:
                continue
            alloc_list = allocation[d]
            if not alloc_list:
                score = 0
            else:
                score = np.mean([C[s, x] for x in alloc_list])
            if score > best_score:
                best_score, best_d = score, d
        if best_d is not None:
            allocation[best_d].add(s)
            remaining[best_d] -= sku_volumes[s]
            allocated.add(s)
            if debug:
                print(f"Assigned SKU {s} to DC_{best_d+1} (avg coappear {best_score:.1f})")

    # Fill remaining capacity
    progress = True
    while progress:
        progress = False
        for d in sorted_dcs:
            cap = remaining[d]
            cand = [s for s in range(n_skus) if s not in allocated and sku_volumes[s] <= cap]
            if not cand:
                continue
            # total coappearance
            if allocation[d]:
                scores = C[list(allocation[d]), :].sum(axis=0)
            else:
                scores = C.sum(axis=0)
            s_best = max(cand, key=lambda s: scores[s])
            allocation[d].add(s_best)
            remaining[d] -= sku_volumes[s_best]
            allocated.add(s_best)
            progress = True
            if debug:
                print(f"DC_{d+1} added SKU {s_best} (score {scores[s_best]:.0f})")

    return allocation, remaining


# IMPORT DATA

In [6]:
df = pd.read_csv(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\DatasetClean.csv')
df['Articolo'] = df['Articolo'].astype(str)
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,Volume cartone,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,33.96900,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,2.66000,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,2.94400,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,2.81600,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,2.54375,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,23.04000,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,25.63600,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,17.02400,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,20.46000,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999


In [7]:
df_vol = df.groupby('Articolo')[['Volume pezzo']].median().reset_index()
df_vol

,Articolo,Volume pezzo
0,100002,0.060000
1,100003,0.060000
2,100004,0.060000
3,100005,0.060000
4,100006,0.060000
...,...,...
18655,99972,0.059375
18656,99973,0.270000
18657,99974,3.312000
18658,99975,0.084000


In [8]:
df_stock = pd.read_excel(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\0.DATA\Giacenza media articoli 2024.xlsx')
df_stock['ARTICOLO'] = df_stock['ARTICOLO'].astype(str)
df_stock = df_stock.groupby('ARTICOLO')[['GIACENZA MEDIA']].mean().reset_index()
df_stock

,ARTICOLO,GIACENZA MEDIA
0,100002,1057.6
1,100003,735.4
2,100004,1072.4
3,100005,1170.0
4,100006,1232.2
...,...,...
20919,99972,19.2
20920,99973,26.5
20921,99974,13.9
20922,99975,7.0


In [9]:
# Perform the join based on 'Articolo'
df_stock_vol = df_stock.merge(df_vol[['Articolo','Volume pezzo']], how='left', left_on='ARTICOLO', right_on='Articolo')

# Drop duplicate column 'ARTICOLO' after merge
df_stock_vol.drop(columns=['Articolo'], inplace=True)

#Compute stock in volum
df_stock_vol['Giacenza Pezzi Volume [m3]'] = df_stock_vol['Volume pezzo'] / 1000
df_stock_vol['Giacenza Pezzi Volume [m3]'] = df_stock_vol['GIACENZA MEDIA'] * df_stock_vol['Giacenza Pezzi Volume [m3]']

#df_stock_vol = df_stock_vol.drop_duplicates()

df_stock_vol

,ARTICOLO,GIACENZA MEDIA,Volume pezzo,Giacenza Pezzi Volume [m3]
0,100002,1057.6,0.060000,0.063456
1,100003,735.4,0.060000,0.044124
2,100004,1072.4,0.060000,0.064344
3,100005,1170.0,0.060000,0.070200
4,100006,1232.2,0.060000,0.073932
...,...,...,...,...
20919,99972,19.2,0.059375,0.001140
20920,99973,26.5,0.270000,0.007155
20921,99974,13.9,3.312000,0.046037
20922,99975,7.0,0.084000,0.000588


In [10]:
df_stock_vol['Giacenza Pezzi Volume [m3]'].sum()

9121.832503351417

In [11]:
# Perform a left join to maintain the original number of rows in df
df = df.merge(df_stock_vol[['ARTICOLO', 'Giacenza Pezzi Volume [m3]']], how='left', left_on='Articolo', right_on='ARTICOLO')

df['Volume evaso [m3]'] = df['Pezzi evasi'] * df['Volume pezzo'] / 1000

# Drop the extra 'ARTICOLO' column from df_stock_vol (after the merge)
df.drop(columns=['ARTICOLO'], inplace=True)

# Ensure no additional duplicates were introduced
#df = df.drop_duplicates()

df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr1,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3]
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Cura Persona,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.140424,0.001823
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.419256,0.004320
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,...,Cura Persona,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.148149,0.002430
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Casa e Bucato,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Casa e Bucato,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Casa e Bucato,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Casa e Bucato,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025


In [12]:
df.drop_duplicates(subset = 'Articolo').groupby('Ecr1')['Giacenza Pezzi Volume [m3]'].sum().sum()

8737.318431127718

# GREEDY ORDER

In [13]:
# 3. Execute the full pipeline

# Assume df and df_stock_vol are already loaded in the environment
R, order_ids, sku_ids = build_R(df)

# Compute SKU volumes (m3) from df_stock_vol
vol_series = df_stock_vol.set_index('ARTICOLO')['Giacenza Pezzi Volume [m3]']
median_vol = vol_series.median()
vol_series_filled = vol_series.fillna(median_vol)
sku_volumes = np.array([vol_series_filled.get(sku, median_vol) for sku in sku_ids], dtype=float)

# Define DC capacities (m3)
capacity_A = 25000 * 0.37  # replace with actual if needed
capacity_B =  6800 * 0.37
k_volume = np.array([capacity_A, capacity_B])

# Run heuristic with debug on
allocation, remaining_volume = greedy_bestsellers_volume(R, k_volume, sku_volumes, debug=True)
print("\nFinal remaining volumes (m3):", remaining_volume)

# 4. Map indices back to Articolo codes
alloc_map = []
for d, skus in enumerate(allocation):
    for s in skus:
        alloc_map.append({
            'Warehouse': f'DC_{d+1}', 
            'Articolo': sku_ids[s]
        })
df_allocation = pd.DataFrame(alloc_map)

Initial B = 5384
  DC_2 seeded 374 SKUs (k_card < B)
  Recomputed B = 0
B <= 0, falling back to GREEDY_SEEDS
Seed DC_1 with SKU 2127
No seed candidates for DC_2; skipping
Assigned SKU 1701 to DC_1 (avg coappear 49.0)
Assigned SKU 388 to DC_1 (avg coappear 58.5)
Assigned SKU 746 to DC_1 (avg coappear 48.0)
Assigned SKU 787 to DC_1 (avg coappear 38.0)
Assigned SKU 3088 to DC_1 (avg coappear 59.0)
Assigned SKU 2264 to DC_1 (avg coappear 53.2)
Assigned SKU 8805 to DC_1 (avg coappear 56.4)
Assigned SKU 967 to DC_1 (avg coappear 65.0)
Assigned SKU 2271 to DC_1 (avg coappear 51.0)
Assigned SKU 432 to DC_1 (avg coappear 34.5)
Assigned SKU 885 to DC_1 (avg coappear 46.7)
Assigned SKU 600 to DC_1 (avg coappear 51.7)
Assigned SKU 3939 to DC_1 (avg coappear 38.7)
Assigned SKU 16610 to DC_1 (avg coappear 49.6)
Assigned SKU 6884 to DC_1 (avg coappear 29.0)
Assigned SKU 5164 to DC_1 (avg coappear 36.4)
Assigned SKU 1352 to DC_1 (avg coappear 44.3)
Assigned SKU 1263 to DC_1 (avg coappear 33.4)
Assigne

In [14]:
df_allocation

,Warehouse,Articolo
0,DC_1,3456
1,DC_1,7199
2,DC_1,43556
3,DC_1,50045
4,DC_1,74657
...,...,...
18655,DC_2,45970
18656,DC_2,98902
18657,DC_2,3618
18658,DC_2,101309


# STATISTICS

In [15]:
df = pd.merge(df, df_allocation, on='Articolo', how='left')
df['Warehouse'] = df['Warehouse'].replace({'DC_1': 'A', 'DC_2': 'B'})
df

,Unnamed: 0,Num. Ordine,Mese-Giorno,Articolo,Descrizione,Pezzi ordinati,Pezzi evasi,Pz x CT,Pz x TH,Volume pezzo,...,Ecr2,Ecr3,Ecr4,Canale,Cliente,PV,Percorso,Giacenza Pezzi Volume [m3],Volume evaso [m3],Warehouse
0,0,738378,04-18,20918,CAREFREE COTTON SALVASLIP 44 PZ.DISTESO,6,6,24,0,1.288000,...,Igienico Sanitari,Assorbenti,Salvaslip e Proteggislip,Piume Diretti,2104490,SM,26,3.474766,0.007728,B
1,1,738379,04-18,3456,STUDIO L.5 INVISI FIX GEL FOR.LIQ.150ml,6,6,6,0,0.378000,...,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.206955,0.002268,A
2,2,738379,04-18,7199,STUDIO L.8 FIX&FORCE GEL IPERFOR.150 ML.,6,6,6,0,0.303750,...,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.140424,0.001823,A
3,3,738379,04-18,43556,STUDIO L.9 INDESTRUC.GEL ESTREMO 150 ML,6,6,6,0,0.720000,...,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.419256,0.004320,A
4,4,738379,04-18,50045,STUDIO L.5 INVISI FIX GEL CR.FOR.VAS.150,6,6,6,0,0.405000,...,Capelli,Styling Capelli,Gel e Cere Capelli,Piume Diretti,2104490,SM,26,0.148149,0.002430,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447292,2447292,791134,12-29,92720,CHANTECLAIR SGRASS.600 MLx2 PZ.LIMONE,1,1,6,0,3.271125,...,Superfici,Detergenti Superfici,Sgrassatori,B2C,2152842,UD,999,2.330022,0.003271,A
2447293,2447293,791134,12-29,98819,VIM GEL BAGNO 5in1 ANTICALCARE 1000 ML,1,1,12,0,1.559250,...,Bagno,Detergenti Bagno e WC,Detergenti Bagno,B2C,2152842,UD,999,0.943970,0.001559,A
2447294,2447294,791134,12-29,105877,CIF CREMA GREEN ACTIVE 500 ML PINK BLOOM,1,1,16,0,0.728000,...,Superfici,Detergenti Superfici,Detergenti Multiuso,B2C,2152842,UD,999,0.743142,0.000728,A
2447295,2447295,791134,12-29,107171,OMINO B.DET.IDROCAPS SALVAFIBRE 20pz,1,1,8,0,2.025000,...,Bucato,Detersivi Bucato,Detersivi Capsule Lavatrice,B2C,2152842,UD,999,0.532170,0.002025,A


In [16]:
df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [],
        'Magazzino B':[],
        
        'Codici in A':[],
        'Codici in B':[],
        
        'Stock [m3] in A':[],
        'Stock [m3] in B':[],
        
        '% Ordini completati in AB':[],
        '% Ordini completati in A':[], 
        '% Ordini completati in B':[],

        'Vol[m3] Ordini completati in A':[], 
        'Vol[m3] Ordini completati in B':[],
        'Vol[m3] Ordini completati in AB':[],
        'Vol[m3] Ordini completati in AB (A)':[],
        'Vol[m3] Ordini completati in AB (B)':[],
        
        '% Rotte completate in AB' :[],
        '% Rotte completate in A' :[],
        '% Rotte completate in B' :[],

        'Vol[m3] Rotte completati in A' :[],
        'Vol[m3] Rotte completati in B' :[],
        'Vol[m3] Rotte completati in AB' :[],
        'Vol[m3] Rotte completati in AB (A)' :[],
        'Vol[m3] Rotte completati in AB (B)' :[],

    }
)
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,Vol[m3] Ordini completati in A,...,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),% Rotte completate in AB,% Rotte completate in A,% Rotte completate in B,Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)


In [17]:
assignment_A = df.groupby('Warehouse')['Articolo'].unique().get('A', 0)
assignment_B = df.groupby('Warehouse')['Articolo'].unique().get('B', 0)

code_A = df.groupby('Warehouse')['Articolo'].nunique().get('A', 0)
code_B = df.groupby('Warehouse')['Articolo'].nunique().get('B', 0)
# print('Article division', article_division)

#Order analysis
order_grouped_df = df.groupby(['Mese-Giorno','Num. Ordine']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

order_movment_A = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100
order_movment_B = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(order_grouped_df) * 100
order_movment_AB = len(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(order_grouped_df) * 100

order_vol_A = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
order_vol_B = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
order_vol_AB = order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

AB_order_list = list(order_grouped_df[order_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & order_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Num. Ordine'])
df_AB_order_volume = df.groupby(['Num. Ordine','Warehouse'])[['Volume evaso [m3]']].sum().reset_index()
df_AB_order_volume = df_AB_order_volume[df_AB_order_volume['Num. Ordine'].isin(AB_order_list)].groupby('Warehouse')['Volume evaso [m3]'].sum()

#Route analysis
route_grouped_df = df.groupby(['Mese-Giorno','Percorso']).agg({
    'Warehouse': lambda x: list(x.unique()),  # Stores unique warehouses as lists
    'Volume evaso [m3]': 'sum'  # Sums up volume
}).reset_index()

route_movment_A = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100
route_movment_B = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]) / len(route_grouped_df) * 100
route_movment_AB = len(route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]) / len(route_grouped_df) * 100

route_vol_A = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum() 
route_vol_B = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'") & ~route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'")]['Volume evaso [m3]'].sum() 
route_vol_AB = route_grouped_df[route_grouped_df['Warehouse'].astype(str).str.contains(r"'A'") & route_grouped_df['Warehouse'].astype(str).str.contains(r"'B'")]['Volume evaso [m3]'].sum()

# Step 1: Extract valid routes containing both 'A' and 'B' in 'Warehouse'
AB_order_list = route_grouped_df[
    route_grouped_df['Warehouse'].astype(str).str.contains(r'A') & 
    route_grouped_df['Warehouse'].astype(str).str.contains(r'B')
][['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).tolist()

# Step 2: Compute total volume for each route
df_AB_route_volume = df.groupby(['Mese-Giorno', 'Percorso', 'Warehouse'])[['Volume evaso [m3]']].sum().reset_index()

# Step 3: Filter only the relevant routes and sum by Warehouse
df_AB_route_volume = df_AB_route_volume[
    df_AB_route_volume[['Mese-Giorno', 'Percorso']].apply(tuple, axis=1).isin(AB_order_list)
].groupby('Warehouse')['Volume evaso [m3]'].sum()


weighted_stock = df.groupby(['Articolo','Warehouse',])['Giacenza Pezzi Volume [m3]'].mean() * (df.groupby(['Articolo','Warehouse',])['Pezzi evasi'].sum() / df.groupby(['Articolo'])['Pezzi evasi'].sum())
stock_A = weighted_stock.groupby('Warehouse').sum().get('A', 0)
stock_B = weighted_stock.groupby('Warehouse').sum().get('B', 0)

df_results = pd.DataFrame.from_dict(
    {
        'Magazzino A': [assignment_A],
        'Magazzino B': [assignment_B],
        
        'Codici in A':[code_A],
        'Codici in B':[code_B],
        
        'Stock [m3] in A':[stock_A],
        'Stock [m3] in B':[stock_B],
        
        '% Ordini completati in AB':[order_movment_AB],
        '% Ordini completati in A':[order_movment_A],
        '% Ordini completati in B':[order_movment_B],

        '% Rotte completate in AB':[route_movment_AB],
        '% Rotte completate in A':[route_movment_A],
        '% Rotte completate in B':[route_movment_B],

        'Vol[m3] Ordini completati in A':[order_vol_A],
        'Vol[m3] Ordini completati in B':[order_vol_B],
        'Vol[m3] Ordini completati in AB':[order_vol_AB],
        'Vol[m3] Ordini completati in AB (A)':[df_AB_order_volume.get('A', 0)],
        'Vol[m3] Ordini completati in AB (B)':[df_AB_order_volume.get('B', 0)],


        'Vol[m3] Rotte completati in A':[route_vol_A],
        'Vol[m3] Rotte completati in B':[route_vol_B],
        'Vol[m3] Rotte completati in AB':[route_vol_AB],
        'Vol[m3] Rotte completati in AB (A)' :[df_AB_route_volume.get('A', 0)],
        'Vol[m3] Rotte completati in AB (B)' :[df_AB_route_volume.get('B', 0)],

        
    }
)

In [18]:
df_results

,Magazzino A,Magazzino B,Codici in A,Codici in B,Stock [m3] in A,Stock [m3] in B,% Ordini completati in AB,% Ordini completati in A,% Ordini completati in B,% Rotte completate in AB,...,Vol[m3] Ordini completati in A,Vol[m3] Ordini completati in B,Vol[m3] Ordini completati in AB,Vol[m3] Ordini completati in AB (A),Vol[m3] Ordini completati in AB (B),Vol[m3] Rotte completati in A,Vol[m3] Rotte completati in B,Vol[m3] Rotte completati in AB,Vol[m3] Rotte completati in AB (A),Vol[m3] Rotte completati in AB (B)
0,"[3456, 7199, 43556, 50045, 74657, 27576, 27577...","[20918, 96352, 104170, 80786, 73455, 96863, 65...",18286,374,6604.762836,2515.955825,60.044181,36.414841,3.540978,87.9086,...,2263.927389,582.312812,42308.993196,24549.323818,17759.669378,427.516714,41.04535,44686.671333,26385.734493,18300.93684


In [19]:
df_results.to_excel(r'C:\Users\Matteo.Gabellini\OneDrive - Alma Mater Studiorum Università di Bologna\DOTTORATO\1.RICERCA\0.CONFERENCE PAPER\6.ICIL\1.WAREHOUSE ALLOCATION\3.RESULTS\CATELAN_4.xlsx')